In [4]:
## Load the dataset
with open("/Users/praroopchanda/Desktop/LLM_Scratch/the-verdict.txt","r",encoding="utf-8") as f:
    raw_text=f.read()

print("the total numbers of character:",len(raw_text))
print(raw_text[:99])

the total numbers of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
import re
text="Hello, world. Is this-- a test?"
result=re.split(r'([,.:;?_!"()\']|--|\s)',text) ## splitting based on these characters
result=[item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [6]:
## splitting the raw_text based on the above characters
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
print(len(preprocessed)) ## total number of tokens for verdict dataset

4690


## Step:2 Now Converting it into Token ID's

In [8]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)
print(vocab_size)

1130


In [9]:
vocab={token:integer for integer,token in enumerate(all_words)}

In [10]:
for i,token in enumerate(vocab.items()):
    print(token)
    if i==20:
        break

## at the time of decoding we also need to build a {token_id:token} dictionary as well --> reverse mapping
    

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [11]:
## building the tokenizer class for encoder and decoder
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int=vocab ## vocab is basically a dictionary: Mapping from string text to Token ID (Encoder)
        self.int_to_str={i:s for s,i in vocab.items()} # building reverse mapping from TokenID to text (Decoder)

    def encode(self,text):
        preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed=[
            item.strip() for item in preprocessed if item.strip()
        ]  ## removing white space from tokens

        ids=[self.str_to_int[s] for s in preprocessed]

        return ids ## list of token ID for the given text
    
    def decode(self,ids): # convert from token ids to string text back
        text=" ".join([self.int_to_str[i] for i in ids])
        ## replacing space before punctuation marks
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
      

In [12]:
## testing our tokenizer for encoder and decoder
tokenizer=SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

ids=tokenizer.encode(text)

print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [15]:
## now decoding --> converting back from ids to text
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
### what if we give something that is not in our vocubalry -> we will get error
text="hello how are you?"
ids=tokenizer.encode(text)
print(ids)

## To prevent such things we are using Special Tokens

In [53]:
## extenting the vocabulary by addding 2 additional tokens
all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab={token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


In [54]:
print(list(vocab.items())[-5:len(vocab)])

[('younger', 1127), ('your', 1128), ('yourself', 1129), ('<|endoftext|>', 1130), ('<|unk|>', 1131)]


In [55]:
## building the updated tokenizer class with logic to handle the missing tokens
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int=vocab ## Mapping from Text to Token Id (encoder)
        self.int_to_str={i:s for s,i in vocab.items()} # reverse mapping from Token ID to Text (decoder)

    def encode(self,text):
        preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text) # split the text into words or tokens
        preprocessed=[item.strip() for item in preprocessed if item.strip()] # removing white spaces as tokens
        '''Now handle the unknown tokes as well, replace them with "<|unk|>" '''
        preprocessed=[item if item in self.str_to_int
                      else "<|unk|>" for item in preprocessed]
        ids=[self.str_to_int[s] for s in preprocessed]

        return ids
    
    def decode(self,ids):
        text=" ".join([self.int_to_str[i] for i in ids])
        ## replace space before punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [62]:
## testing the new tokenizer
tokenizer=SimpleTokenizerV2(vocab)

text1="hello, do you like tea?"
text2="In the sunlit terraces of the palace."

text= " <|endoftext|> ".join((text1,text2))

print(text)

hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [63]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [64]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

### Other tokens [BOS], [EOS], [PAD] --> for varying length in batches
### GPT only uses [|endoftext|] token , not even unk token --> which is basically after end of one document or text source
### GPT uses byte pair encoding , thats how it deals with unknown tokens